## Prérequis

- Python 3.9+
- pandas
- pyarrow (pour la lecture des fichiers Parquet)

> ⚠️ **Important** : Ce notebook est uniquement pertinent pour les datasets qui ont été chunkés **sans overlap**. Si vos chunks ont du texte qui se chevauche (par ex. 50 tokens de chevauchement entre chunks consécutifs), la reconstruction des documents par simple concaténation résultera en **du texte dupliqué** dans la sortie. Pour les chunks avec overlap, une stratégie de déduplication plus sophistiquée serait nécessaire.

In [ ]:
# Installation des dépendances si nécessaire
# !pip install pandas pyarrow

## Structure des données d'entrée

Les fichiers Parquet d'entrée contiennent généralement les colonnes suivantes :

| Colonne | Description |
|---------|-------------|
| `chunk_id` | Identifiant unique du chunk |
| `doc_id` | Identifiant du document original |
| `chunk_index` | Index du chunk dans le document (pour l'ordre) |
| `text` | Contenu textuel du chunk |
| `chunk_text` | *(Obsolète)* Texte du chunk formaté |
| `embeddings_bge-m3` | *(Obsolète)* Vecteur d'embedding du chunk |
| ... | Autres métadonnées spécifiques au corpus |

## Étape 1 : Importer les bibliothèques nécessaires

In [1]:
import pandas as pd
from pathlib import Path
import glob

## Étape 2 : Charger les fichiers Parquet

In [ ]:
def load_parquet_files(folder_path: str) -> pd.DataFrame:
    """
    Charge tous les fichiers Parquet d'un dossier et les concatène.
    
    Args:
        folder_path: Chemin vers le dossier contenant les fichiers Parquet
        
    Returns:
        DataFrame contenant toutes les données des fichiers Parquet
    """
    parquet_files = glob.glob(f"{folder_path}/*.parquet")
    
    if not parquet_files:
        raise FileNotFoundError(f"Aucun fichier Parquet trouvé dans {folder_path}")
    
    print(f"{len(parquet_files)} fichier(s) Parquet trouvé(s)")
    
    dfs = []
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        print(f"  ✓ {Path(file_path).name}: {len(df)} lignes")
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal: {len(combined_df)} chunks chargés")
    
    return combined_df

In [ ]:
# ⚠️ Modifiez ce chemin pour pointer vers votre dossier de fichiers Parquet
parquet_folder = "../../data/parquet/mon_corpus"

# Charger les fichiers
df_chunks = load_parquet_files(parquet_folder)

# Afficher un aperçu
df_chunks.head()

In [ ]:
# Afficher les colonnes disponibles
print("Colonnes disponibles:")
for col in df_chunks.columns:
    print(f"  - {col}")

## 🚀 Étape 3 : Supprimer les colonnes obsolètes

In [ ]:
def remove_obsolete_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Supprime les colonnes obsolètes du DataFrame.
    
    Args:
        df: DataFrame contenant les données chunkées
        
    Returns:
        DataFrame sans les colonnes obsolètes
    """
    obsolete_columns = ["chunk_text", "embeddings_bge-m3"]
    
    # Filtrer les colonnes qui existent réellement
    columns_to_drop = [col for col in obsolete_columns if col in df.columns]
    
    if columns_to_drop:
        df = df.drop(columns=columns_to_drop)
        print(f"Colonnes supprimées: {columns_to_drop}")
    else:
        print("ℹAucune colonne obsolète trouvée")
    
    return df

In [ ]:
# Supprimer les colonnes obsolètes
df_cleaned = remove_obsolete_columns(df_chunks)

print(f"\nColonnes restantes: {list(df_cleaned.columns)}")

## 🚀 Étape 4 : Reconstituer les documents originaux

In [ ]:
def reconstruct_documents(df: pd.DataFrame, text_column: str = "text") -> pd.DataFrame:
    """
    Reconstitue les documents originaux en concaténant les chunks.
    
    Chaque document est reconstitué en ordonnant ses chunks par `chunk_index`
    puis en concaténant leurs textes.
    
    Args:
        df: DataFrame contenant les chunks
        text_column: Nom de la colonne contenant le texte à concaténer
        
    Returns:
        DataFrame avec un document complet par `doc_id`
    """
    # Vérifier que les colonnes nécessaires existent
    required_columns = ["doc_id", "chunk_index", text_column]
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Colonnes manquantes: {missing_columns}")
    
    print(f"Reconstruction de {df['doc_id'].nunique()} documents...")
    
    # Trier par doc_id et chunk_index pour assurer l'ordre correct
    df_sorted = df.sort_values(by=["doc_id", "chunk_index"])
    
    # Identifier les colonnes de métadonnées (exclure les colonnes spécifiques aux chunks ET doc_id car c'est la clé de groupement)
    chunk_specific_cols = ["chunk_id", "chunk_index", "chunk_xxh64", text_column, "doc_id"]
    metadata_cols = [col for col in df.columns if col not in chunk_specific_cols]
    
    # Agrégation : concaténer le texte, garder les métadonnées de la première ligne
    agg_dict = {text_column: lambda x: "\n".join(x.astype(str))}
    for col in metadata_cols:
        agg_dict[col] = "first"
    
    df_reconstructed = df_sorted.groupby("doc_id", as_index=False).agg(agg_dict)
    
    # Réorganiser les colonnes : doc_id en premier, puis métadonnées, puis texte
    final_columns = ["doc_id"] + metadata_cols + [text_column]
    df_reconstructed = df_reconstructed[[col for col in final_columns if col in df_reconstructed.columns]]
    
    print(f"{len(df_reconstructed)} documents reconstitués")
    
    return df_reconstructed

In [ ]:
# Reconstituer les documents
df_documents = reconstruct_documents(df_cleaned)

# Afficher un aperçu
df_documents.head()

In [ ]:
# Vérifier le résultat pour un document
sample_doc_id = df_documents['doc_id'].iloc[0]
print(f"Exemple de document reconstitué (doc_id: {sample_doc_id})")
print("=" * 60)
print(df_documents[df_documents['doc_id'] == sample_doc_id]['text'].iloc[0][:1000])
print("...")

## 🚀 Étape 5 : Sauvegarder le résultat

In [ ]:
def save_reconstructed_documents(
    df: pd.DataFrame, 
    output_path: str, 
    format: str = "parquet",
    rows_per_file: int = 50000,
    compression: str = "zstd"
) -> None:
    """
    Sauvegarde les documents reconstitués, en les divisant en plusieurs fichiers si nécessaire.
    
    Args:
        df: DataFrame contenant les documents reconstitués
        output_path: Chemin de sortie (dossier pour parquet, chemin de fichier pour csv/json)
        format: Format de sortie ("parquet", "csv", ou "json")
        rows_per_file: Nombre cible de lignes par fichier parquet (défaut: 50000)
        compression: Algorithme de compression pour parquet ("zstd", "snappy", "gzip", None)
    """
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    
    if format == "parquet":
        # Créer le dossier de sortie
        output_folder = Path(output_path)
        output_folder.mkdir(parents=True, exist_ok=True)
        
        total_rows = len(df)
        
        if total_rows <= rows_per_file:
            # Export en un seul fichier
            output_file = output_folder / f"{output_folder.name}_part_0.parquet"
            df.to_parquet(
                output_file, 
                index=False, 
                compression=compression,
                engine="pyarrow"
            )
            print(f"Documents sauvegardés dans: {output_file} ({total_rows} lignes)")
        else:
            # Export multi-fichiers
            num_files = (total_rows + rows_per_file - 1) // rows_per_file
            
            for i in range(num_files):
                start_idx = i * rows_per_file
                end_idx = min((i + 1) * rows_per_file, total_rows)
                df_batch = df.iloc[start_idx:end_idx]
                
                output_file = output_folder / f"{output_folder.name}_part_{i}.parquet"
                df_batch.to_parquet(
                    output_file, 
                    index=False, 
                    compression=compression,
                    engine="pyarrow"
                )
                print(f"  Partie {i}: {output_file.name} ({len(df_batch)} lignes)")
            
            print(f"\nTotal: {total_rows} lignes sauvegardées dans {num_files} fichier(s) dans {output_folder}/")
    
    elif format == "csv":
        output_file = f"{output_path}.csv"
        df.to_csv(output_file, index=False)
        print(f"Documents sauvegardés dans: {output_file}")
    
    elif format == "json":
        output_file = f"{output_path}.json"
        df.to_json(output_file, orient="records", force_ascii=False, indent=2)
        print(f"Documents sauvegardés dans: {output_file}")
    
    else:
        raise ValueError(f"Format non supporté: {format}")

In [ ]:
# Modifiez ce chemin pour définir votre dossier/fichier de sortie
output_path = "../../data/output/documents_reconstitues"

# Sauvegarder en Parquet (avec compression ZSTD, divisé en fichiers de 50k lignes)
# Recommandé pour les grands jeux de données
save_reconstructed_documents(
    df_documents, 
    output_path, 
    format="parquet",
    rows_per_file=50000,
    compression="zstd"
)

# Ou en JSON (fichier unique)
# save_reconstructed_documents(df_documents, output_path, format="json")